# 🔍 SHAP Analysis and Visualization

## Interpreting the Thermal Intensity Model

**Author:** Fafa (GitHub: Fateme9977)  
**Institution:** K. N. Toosi University of Technology

---

This notebook performs SHAP analysis to interpret the XGBoost thermal intensity model
and identify key drivers of heating energy consumption.

In [ ]:
# Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import joblib
import warnings
warnings.filterwarnings('ignore')

import shap
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

# Initialize SHAP
shap.initjs()

# Set style
plt.style.use('seaborn-v0_8-whitegrid')

# Project paths
PROJECT_ROOT = Path.cwd().parent
OUTPUT_DIR = PROJECT_ROOT / 'output'
MODELS_DIR = OUTPUT_DIR / 'models'

print(f'SHAP version: {shap.__version__}')

## 1. Load Model and Data

In [ ]:
# Load model
model = joblib.load(MODELS_DIR / 'xgboost_thermal_intensity.joblib')
encoders = joblib.load(MODELS_DIR / 'label_encoders.joblib')

print('Model loaded successfully')
print(f'Number of features: {model.n_features_in_}')

In [ ]:
# Load data
df = pd.read_csv(OUTPUT_DIR / '03_gas_heated_clean.csv')
print(f'Loaded {len(df):,} households')

In [ ]:
# Prepare features (same as training)
numeric_features = ['HDD65', 'A_heated', 'building_age', 'log_sqft']
categorical_features = ['TYPEHUQ', 'YEARMADERANGE', 'DRAFTY', 'ADQINSUL', 
                        'TYPEGLASS', 'REGIONC', 'DIVISION', 'envelope_class', 'climate_zone']

numeric_features = [f for f in numeric_features if f in df.columns]
categorical_features = [f for f in categorical_features if f in df.columns]

X = df[numeric_features + categorical_features].copy()

# Encode categoricals
for col in categorical_features:
    if col in encoders:
        X[col] = encoders[col].transform(X[col].fillna('missing').astype(str))
    elif X[col].dtype == 'object':
        X[col] = pd.factorize(X[col].fillna('missing'))[0]
    else:
        X[col] = X[col].fillna(-1)

# Fill numeric NAs
for col in numeric_features:
    X[col] = X[col].fillna(X[col].median())

print(f'Feature matrix shape: {X.shape}')

## 2. Compute SHAP Values

In [ ]:
# Sample for SHAP calculation (large datasets can be slow)
sample_size = min(2000, len(X))
X_sample = X.sample(n=sample_size, random_state=42)

print(f'Computing SHAP values for {sample_size} samples...')

In [ ]:
# Create SHAP explainer
explainer = shap.TreeExplainer(model)

# Compute SHAP values
shap_values = explainer.shap_values(X_sample)

print(f'SHAP values computed. Shape: {shap_values.shape}')

## 3. Global Feature Importance

In [ ]:
# Summary plot (beeswarm)
plt.figure(figsize=(12, 8))
shap.summary_plot(shap_values, X_sample, show=False, max_display=15)
plt.title('Feature Impact on Thermal Intensity (SHAP Values)', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Bar plot of mean |SHAP|
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values, X_sample, plot_type='bar', show=False, max_display=15)
plt.title('Mean |SHAP| - Feature Importance', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Table of feature importance
mean_abs_shap = np.abs(shap_values).mean(axis=0)
importance_df = pd.DataFrame({
    'Feature': X_sample.columns,
    'Mean |SHAP|': mean_abs_shap,
    'Importance (%)': mean_abs_shap / mean_abs_shap.sum() * 100
}).sort_values('Mean |SHAP|', ascending=False)

print('Feature Importance Ranking (SHAP):')
print(importance_df.to_string(index=False))

## 4. Dependence Plots

In [ ]:
# Top features for dependence plots
top_features = importance_df.head(6)['Feature'].tolist()
print(f'Creating dependence plots for: {top_features}')

In [ ]:
# Create dependence plots for top features
n_plots = min(6, len(top_features))
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

for i, feature in enumerate(top_features[:n_plots]):
    plt.sca(axes[i])
    feature_idx = list(X_sample.columns).index(feature)
    shap.dependence_plot(feature_idx, shap_values, X_sample, show=False, ax=axes[i])
    axes[i].set_title(f'SHAP Dependence: {feature}', fontsize=12)

# Hide unused axes
for i in range(n_plots, len(axes)):
    axes[i].set_visible(False)

plt.tight_layout()
plt.show()

In [ ]:
# HDD65 dependence with interaction
if 'HDD65' in X_sample.columns:
    plt.figure(figsize=(10, 6))
    hdd_idx = list(X_sample.columns).index('HDD65')
    
    # Auto-detect interaction
    shap.dependence_plot(hdd_idx, shap_values, X_sample, show=False)
    plt.title('HDD65 Effect on Thermal Intensity', fontsize=14)
    plt.tight_layout()
    plt.show()

## 5. Individual Explanations

In [ ]:
# Waterfall plot for example households

# Select example indices (high, medium, low intensity)
predictions = model.predict(X_sample)
percentiles = [10, 50, 90]  # Low, medium, high

for p in percentiles:
    threshold = np.percentile(predictions, p)
    idx = np.argmin(np.abs(predictions - threshold))
    
    print(f'\n=== Example: {p}th percentile prediction ====')
    print(f'Predicted value: {predictions[idx]:.2f}')
    
    # Create explanation
    explanation = shap.Explanation(
        values=shap_values[idx],
        base_values=explainer.expected_value,
        data=X_sample.iloc[idx],
        feature_names=X_sample.columns.tolist()
    )
    
    plt.figure(figsize=(10, 6))
    shap.waterfall_plot(explanation, show=False, max_display=10)
    plt.title(f'SHAP Waterfall ({p}th percentile)', fontsize=14)
    plt.tight_layout()
    plt.show()

## 6. Force Plots

In [ ]:
# Force plot for single observation
idx = 0  # First sample
shap.force_plot(
    explainer.expected_value,
    shap_values[idx],
    X_sample.iloc[idx],
    matplotlib=True,
    show=False
)
plt.title('SHAP Force Plot - Single Household', fontsize=12)
plt.tight_layout()
plt.show()

## 7. Analysis by Envelope Class

In [ ]:
# Compare SHAP distributions by envelope class
sample_idx = X_sample.index
df_sample = df.loc[sample_idx].copy()

if 'envelope_class' in df_sample.columns:
    # Mean absolute SHAP by envelope class
    results = []
    
    for env_class in ['poor', 'medium', 'good']:
        mask = (df_sample['envelope_class'] == env_class).values
        if mask.sum() > 10:
            class_shap = np.abs(shap_values[mask]).mean(axis=0)
            for i, feature in enumerate(X_sample.columns):
                results.append({
                    'Envelope Class': env_class,
                    'Feature': feature,
                    'Mean |SHAP|': class_shap[i]
                })
    
    if results:
        class_df = pd.DataFrame(results)
        pivot = class_df.pivot(index='Feature', columns='Envelope Class', values='Mean |SHAP|')
        
        # Heatmap
        plt.figure(figsize=(10, 8))
        sns.heatmap(pivot, annot=True, fmt='.3f', cmap='YlOrRd')
        plt.title('Feature Importance by Envelope Class', fontsize=14)
        plt.tight_layout()
        plt.show()

## 8. Key Physical Insights

In [ ]:
# Interaction: HDD × Draftiness
if 'HDD65' in X_sample.columns and 'DRAFTY' in X_sample.columns:
    plt.figure(figsize=(10, 6))
    hdd_idx = list(X_sample.columns).index('HDD65')
    drafty_idx = list(X_sample.columns).index('DRAFTY')
    
    shap.dependence_plot(
        hdd_idx, shap_values, X_sample,
        interaction_index=drafty_idx,
        show=False
    )
    plt.title('HDD65 Effect × Draftiness Interaction', fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
# Interaction: Building Age × Draftiness
if 'building_age' in X_sample.columns and 'DRAFTY' in X_sample.columns:
    plt.figure(figsize=(10, 6))
    age_idx = list(X_sample.columns).index('building_age')
    drafty_idx = list(X_sample.columns).index('DRAFTY')
    
    shap.dependence_plot(
        age_idx, shap_values, X_sample,
        interaction_index=drafty_idx,
        show=False
    )
    plt.title('Building Age Effect × Draftiness Interaction', fontsize=14)
    plt.tight_layout()
    plt.show()

## 9. Summary of Key Findings

### Top Drivers of Thermal Intensity:
1. **Envelope quality (DRAFTY)**: Drafty homes have significantly higher heating intensity
2. **Climate (HDD65)**: More heating needed in colder climates, but intensity normalized by HDD
3. **Building age**: Older homes tend to have higher intensity due to envelope degradation
4. **Floor area**: Larger homes may have different surface-to-volume ratios

### Key Interactions:
- HDD × Draftiness: Drafty homes suffer more in cold climates
- Age × Draftiness: Older drafty homes are the worst performers

### Implications for Retrofit:
- Air sealing (reducing draftiness) is highly impactful
- Older homes in cold climates are priority targets
- Envelope quality improvement reduces heating intensity substantially

In [ ]:
# Save summary
importance_df.to_csv(OUTPUT_DIR / 'tables' / 'shap_feature_importance_notebook.csv', index=False)
print('Results saved!')